In [1]:
# DESCRIPTION:
# The training images are thresholded to remove any backgrounf noise here.
import cv2
import os
import matplotlib.pyplot as plt
import time
import sys
import numpy as np
import matplotlib.pyplot as plt
import pdb
import scipy.io as sio
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from skimage import measure
from scipy import ndimage
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
from skimage.color import label2rgb

IMDIR='sci'
OUTDIR='sci_th'
if not os.path.isdir(OUTDIR):
    os.mkdir(OUTDIR)
counter=0
for image in os.listdir(IMDIR):
    img = cv2.imread(os.path.join(IMDIR,image))
    _,img=cv2.threshold(img,200,255,cv2.THRESH_BINARY)

    # Writing the result.
    cv2.imwrite(os.path.join(OUTDIR,"{}.png".format(counter)),img)

    # Printing the result so far.
    sys.stdout.write("\rImages Left {}/{}.".format(counter, len(os.listdir(IMDIR))-1))
    sys.stdout.flush()

    # Incrementing the counter
    counter = counter + 1
print("Done with {} images!".format(counter))
# The input images to this api must be binary.

# Main function to perform the splitting..../////////////////////////////////////////////
def line_splitter(input_image):
    # Main list to store the output ligatures i.e. a list of numpy arrays.
    out_image_array=[]

    # Thresholding the images
 #   _, input_image = cv2.threshold(input_image, 5, 255, cv2.THRESH_BINARY)

    # Variable for storing the total number of lines.
    total_lines = 0

    # Extracting the dimensions of the image
    height,width=input_image.shape

    # Making a bool type filter
    filter=np.ones(shape=(1,width),dtype=bool)

    # Converting the source image to bool.
    input_image=input_image.astype(bool)


    # Some workplace variables for the following for loop.
    row=0   #This variable indexs the pixels of the image row wise.
    first_row=True
    line_detected=False

    # This for loop iterates through each pixel wide line of the page and separates them.
    for i in input_image:
        i=np.reshape(i,newshape=(1,width))
        res=np.bitwise_and(i,filter)
        res=np.bitwise_and.reduce(res,axis=1)

        if res==False:
            line_detected=True
            if first_row==True:
                row_start=row
                first_row=False
            row = row + 1
            continue

        if line_detected==True:
            row_end=row
            line_detected=False
            first_row = True
            out_image=input_image[row_start:row_end,:]
            out_image=out_image.astype(int)  #Converting back to int.
            out_image=out_image*255          #Replacing the 1's by 255's.
            # Writing the output.
            out_image_array.append(out_image)
            total_lines=total_lines+1
        row = row + 1
    # print("Lines separated successfully. Total lines {}.".format(total_lines))
    # Returning the list..
    return out_image_array

# Testing the function/////////////////////////////////////////////////////////////////////////

# Selecting the output directory.
OUTDIR='sci_l'
if not os.path.isdir(OUTDIR):
    os.mkdir(OUTDIR)

# Selecting the source directory
IMDIR='sci_th'

# list to store the results.
list_of_lines=[]

# Making a counter to keep track of the images.
counter=0

# Function to sort images in the image directory
def file_sorter(element):
    e=int(element[:-4])
    return e

# The main for loop iterating through each image.
for image in sorted(os.listdir(IMDIR),key=file_sorter):
    digitized_image = cv2.imread(os.path.join(IMDIR, image), cv2.IMREAD_GRAYSCALE)
    list_of_lines=line_splitter(digitized_image)

    for line in list_of_lines:
        cv2.imwrite(os.path.join(OUTDIR, "{}.png".format(counter)), line)
        counter = counter + 1
    # Printing the result so far.
    sys.stdout.write("\rImages Left {}/{}.".format(image[:-4], len(os.listdir(IMDIR))))
    sys.stdout.flush()

    
# The input images to this file must be line separated and binary.

# Main function for performing the splitting. Just takes the line image and separates the ligatures.
def splitter(input_image):
    # Main list to store the output ligatures i.e. a list of numpy arrays.
    out_image_array=[]
    # Variable for storing the total number of ligatures.
    total_ligatures=0
    # Pre-processing the image
   # _, input_image = cv2.threshold(input_image, 5, 255, cv2.THRESH_BINARY)
    
    # Flipping the image counter clockwise.
    input_image = cv2.transpose(input_image)
    input_image = cv2.flip(input_image, flipCode=0)

    # Extracting the dimensions of the image
    height, width = input_image.shape

    # Making a bool type filter
    filter = np.ones(shape=(1, width), dtype=bool)

    # Converting the source image to bool.
    input_image = input_image.astype(bool)

    # Some workplace variables for the following for loop.
    row = 0  # This variable indexs the pixels of the image row wise.
    first_row = True
    ligature_detected = False

    # This for loop iterates through each pixel wide line of the line and separates the ligatures.
    for i in input_image:
        i = np.reshape(i, newshape=(1, width))
        # Applying the operation of the filter with the row.
        res = np.bitwise_and(i, filter)
        # Reducing the result.
        res = np.bitwise_and.reduce(res, axis=1)

        if res == False:
            ligature_detected = True
            if first_row == True:
                row_start = row
                first_row = False
            row = row + 1
            continue

        if ligature_detected == True:
            row_end = row
            ligature_detected = False
            first_row = True
            out_image = input_image[row_start:row_end, :]  # Cropping the image.
            out_image = out_image.astype(int)  # Converting back to int.
            out_image = out_image * 255  # Replacing the 1's by 255's
            # Flipping the output image back in clockwise direction.
            out_image = cv2.transpose(out_image)
            out_image = cv2.flip(out_image, flipCode=+1)
            # Writing the output to the array
            out_image_array.append(out_image)
            total_ligatures=total_ligatures+1

        row = row + 1
    # print("Ligatures separated successfully. Total ligatures {}.".format(total_ligatures))
    # Returning the list..
    return out_image_array



# Executing the function:*****************************************************

# Selecting the output directory.
OUTDIR='sci_lig'
if not os.path.isdir(OUTDIR):
    os.mkdir(OUTDIR)
# Selecting the source directory
IMDIR='sci_l'

# The function returns a list of ligatures. Making that list:
ligs=[]

# This increments for each ligature image separated
counter=0

# Function to sort images in the image directory
def file_sorter(element):
    e=int(element[:-4])
    return e

# Main for loop iterating through the images.
for image in sorted(os.listdir(IMDIR), key=file_sorter):
    # Reading the image.
    digitized_image = cv2.imread(os.path.join(IMDIR, image), cv2.IMREAD_GRAYSCALE)
    ligs=splitter(input_image=digitized_image)

    for l in ligs:
        # Writing the output.
        cv2.imwrite(os.path.join(OUTDIR, "{}.png".format(str(counter))), l)
        counter=counter+1

    # Printing the result so far.
    sys.stdout.write("\rImages Left {}/{}.".format(image[:-4], len(os.listdir(IMDIR))))
    sys.stdout.flush()
    

def stretcher(input_image,degree_of_stretch):
    # Getting the dimensions
    height_original, width_original = input_image.shape
    out_image_array=[]
    # Making the height of the image divisible by degree_of_stretch
    while(height_original%degree_of_stretch!=0):
        height_original=height_original+1
        input_image=np.insert(input_image,0,255,0)


    # degree_of_stretch:
    # How many pixels to skip for the shifting operation.
    # OR we can say that it is the degree of stretch.
    final_width = width_original + int(height_original / degree_of_stretch)
    img_modified = np.ones((height_original, final_width))
    img_modified=img_modified*255
    counter = 0
    inserts = 0    #paddings
    for row in img_modified:
        if counter % degree_of_stretch == 0:
            inserts = inserts + 1
        row[0 + inserts: width_original + inserts] = input_image[counter] #  row[2:22] replace by input_image[1]
        counter = counter + 1
    out_image_array.append(img_modified)
    #return img_modified
    return out_image_array

IMDIR='sci_lig'

# Selecting the output directory for small ligatures.
OUTDIR='sci_str'
if not os.path.isdir(OUTDIR):
    os.mkdir(OUTDIR)
    
list_of_stretched=[]

# Making a counter to keep track of the images.
counter=0

# Function to sort images in the image directory
def file_sorter(element):
    e=int(element[:-4])
    return e

# Main for loop to rotate the images./////////////////////////////////////////////////////////////////////
for image in os.listdir(IMDIR):
    # Reading the input image.
    img_original = cv2.imread(os.path.join(IMDIR, image), cv2.IMREAD_GRAYSCALE)
    #tasveer1=stretcher(img_original,2)
   # list_of_stretched=deStretcher(tasveer1,2)
    list_of_stretched=stretcher(img_original,2)
    
    for ligatures in list_of_stretched:
        cv2.imwrite(os.path.join(OUTDIR, "{}.png".format(counter)), ligatures)
        counter = counter + 1
    # Printing the result so far.
    sys.stdout.write("\rImages Left {}/{}.".format(image[:-4], len(os.listdir(IMDIR))))
    sys.stdout.flush()

#MAINNN 


IMDIR='sci_str'
OUTDIR ='sci_d'
OUTDIRo='sci_nd'
counter=0
dilated_image=[]
for image in os.listdir(IMDIR):
    input_image =cv2.imread(os.path.join(IMDIR,image),cv2.IMREAD_UNCHANGED)
    connectivity=8
    blur_radius =0.6
    threshold = 100
    extent=[]
    out_image_array=[]
    total_ligatures=0
    transposed_image = cv2.transpose(input_image)
    rotated_image = cv2.flip(transposed_image, flipCode = 0)
    input_image = rotated_image
    imgf = ndimage.gaussian_filter(input_image, blur_radius)
    threshold = 100
    input_image = (imgf < threshold).astype(np.uint8)
    nlabels,labels, stats, centroids = cv2.connectedComponentsWithStats(input_image)
   # plt.imshow(labels)
  #  plt.show()
    stats=stats.astype(np.uint16)
    LABEL_ARRAY = np.arange(1, nlabels)
    n = 0
    for region in regionprops(labels):
        n+=1
        if n <=1 and region.extent > 0.51:
            width = stats[n, cv2.CC_STAT_WIDTH]
            height = stats[n, cv2.CC_STAT_HEIGHT]
            x = stats[n, cv2.CC_STAT_LEFT]
            y = stats[n, cv2.CC_STAT_TOP]
            roi = input_image[y:y+height+10,x:x+width]
           # print("Component before dilation")
           # plt.imshow(roi)
          #  plt.show()
            kernel=cv2.getStructuringElement(cv2.MORPH_CROSS,(4,4))
            roi[:]=cv2.dilate(roi, kernel, iterations=1) 
           # print("Component after dilation")
          #  plt.imshow(roi)
           # plt.show()
         #  input_image[:]=cv2.dilate(input_image, kernel, iterations=1)
           # print("Component before dilation")
           # plt.imshow(roi)
          #  plt.show()
         #   kernel=cv2.getStructuringElement(cv2.MORPH_CROSS,(4,3))
         #   roi[:]=cv2.dilate(roi, kernel, iterations=1)      
            out_image = cv2.transpose(input_image)
            out_image = cv2.flip(out_image, flipCode=+1)
            out_image_array.append(out_image)
            plt.imsave(os.path.join(OUTDIR, "{}.png".format(str(counter))),out_image)
        elif n <=1 and region.extent < 0.52:
            in_image = cv2.transpose(input_image)
            in_image = cv2.flip(in_image, flipCode=+1)
            plt.imsave(os.path.join(OUTDIRo, "{}.png".format(counter)), in_image)
    counter=counter+1
    # Printing the result so far.
    sys.stdout.write("\rImages Left {}/{}.".format(image[:-4], len(os.listdir(IMDIR))))
    sys.stdout.flush()

Images Left 146/146.Done with 147 images!
Images Left 9999/60166..

In [3]:
# The input images to this file must be line separated and binary.
import cv2
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import sys

# Main function for performing the splitting. Just takes the line image and separates the ligatures.
def splitter(input_image):
    # Main list to store the output ligatures i.e. a list of numpy arrays.
    out_image_array=[]
    # Variable for storing the total number of ligatures.
    total_ligatures=0
    # Pre-processing the image
    #_, input_image = cv2.threshold(input_image, 50, 255, cv2.THRESH_BINARY)
    
    # Flipping the image counter clockwise.
    input_image = cv2.transpose(input_image)
    input_image = cv2.flip(input_image, flipCode=0)

    # Extracting the dimensions of the image
    height, width = input_image.shape

    # Making a bool type filter
    filter = np.full((1, width),30)

    # Converting the source image to bool.
    #input_image = input_image.astype(bool)

    # Some workplace variables for the following for loop.
    row = 0  # This variable indexs the pixels of the image row wise.
    first_row = True
    ligature_detected = False

    # This for loop iterates through each pixel wide line of the line and separates the ligatures.
    for i in input_image:
        i = np.reshape(i, newshape=(1, width))
        # Applying the operation of the filter with the row.
        res = np.subtract(i, filter)
        # Reducing the result.
        res = np.subtract.reduce(res, axis=1)

        if res !=0:
            ligature_detected = True
            if first_row == True:
                row_start = row
                first_row = False
            row = row + 1
            continue

        if ligature_detected == True:
            row_end = row
            ligature_detected = False
            first_row = True
            out_image = input_image[row_start:row_end, :]  # Cropping the image.
          #  out_image = out_image.astype(int)  # Converting back to int.
          #  out_image = out_image * 255  # Replacing the 1's by 255's
            # Flipping the output image back in clockwise direction.
            out_image = cv2.transpose(out_image)
            out_image = cv2.flip(out_image, flipCode=+1)
            # Writing the output to the array
            out_image_array.append(out_image)
            total_ligatures=total_ligatures+1

        row = row + 1
    print("Ligatures separated successfully. Total ligatures {}.".format(total_ligatures))
    # Returning the list..
    return out_image_array
# Executing the function:*****************************************************
# Selecting the output directory.
OUTDIR='sci_nd_split'
if not os.path.isdir(OUTDIR):
    os.mkdir(OUTDIR)
# Selecting the source directory
IMDIR='sci_nd'
# The function returns a list of ligatures. Making that list:
ligs=[]
# This increments for each ligature image separated
counter=0
# Function to sort images in the image directory
def file_sorter(element):
    e=int(element[:-4])
    return e
# Main for loop iterating through the images.
for image in sorted(os.listdir(IMDIR), key=file_sorter):
    # Reading the image.
    digitized_image = cv2.imread(os.path.join(IMDIR, image),0)
    ligs=splitter(digitized_image)

    for l in ligs:
        # Writing the output.
        cv2.imwrite(os.path.join(OUTDIR, "{}.png".format(str(counter))), l)
        counter=counter+1

    # Printing the result so far.
    sys.stdout.write("\rImages Left {}/{}.".format(image[:-4], len(os.listdir(IMDIR))))
    sys.stdout.flush()

Ligatures separated successfully. Total ligatures 1.
Images Left 0/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 2/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 5/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 8/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 9/38455.Ligatures separated successfully. Total ligatures 3.
Images Left 12/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 13/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 15/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 16/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 17/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 18/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 19/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 20/38455.Ligatures separated successfully. Total lig

Images Left 171/38455.Ligatures separated successfully. Total ligatures 2.
Images Left 172/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 174/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 176/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 180/38455.Ligatures separated successfully. Total ligatures 2.
Images Left 181/38455.Ligatures separated successfully. Total ligatures 3.
Images Left 182/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 188/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 189/38455.Ligatures separated successfully. Total ligatures 3.
Images Left 190/38455.Ligatures separated successfully. Total ligatures 2.
Images Left 191/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 193/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 195/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 196/38455.Lig

Images Left 531/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 533/38455.Ligatures separated successfully. Total ligatures 3.
Images Left 534/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 535/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 536/38455.Ligatures separated successfully. Total ligatures 2.
Images Left 538/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 539/38455.Ligatures separated successfully. Total ligatures 3.
Images Left 540/38455.Ligatures separated successfully. Total ligatures 2.
Images Left 541/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 542/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 545/38455.Ligatures separated successfully. Total ligatures 4.
Images Left 546/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 547/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 548/38455.Lig

Images Left 842/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 844/38455.Ligatures separated successfully. Total ligatures 2.
Images Left 846/38455.Ligatures separated successfully. Total ligatures 5.
Images Left 847/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 848/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 849/38455.Ligatures separated successfully. Total ligatures 3.
Images Left 850/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 851/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 852/38455.Ligatures separated successfully. Total ligatures 2.
Images Left 853/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 856/38455.Ligatures separated successfully. Total ligatures 2.
Images Left 858/38455.Ligatures separated successfully. Total ligatures 2.
Images Left 861/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 862/38455.Lig

Images Left 1167/38455.Ligatures separated successfully. Total ligatures 4.
Images Left 1168/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 1169/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 1173/38455.Ligatures separated successfully. Total ligatures 2.
Images Left 1175/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 1177/38455.Ligatures separated successfully. Total ligatures 6.
Images Left 1179/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 1181/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 1183/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 1184/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 1185/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 1186/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 1188/38455.Ligatures separated successfully. Total ligatures 3.
Images Left 

...

Images Left 59323/38455.Ligatures separated successfully. Total ligatures 2.
Images Left 59325/38455.Ligatures separated successfully. Total ligatures 3.
Images Left 59326/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 59329/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 59330/38455.Ligatures separated successfully. Total ligatures 2.
Images Left 59331/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 59332/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 59333/38455.Ligatures separated successfully. Total ligatures 3.
Images Left 59335/38455.Ligatures separated successfully. Total ligatures 2.
Images Left 59336/38455.Ligatures separated successfully. Total ligatures 2.
Images Left 59337/38455.Ligatures separated successfully. Total ligatures 2.
Images Left 59338/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 59339/38455.Ligatures separated successfully. Total ligatures 1.

Images Left 59482/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 59483/38455.Ligatures separated successfully. Total ligatures 4.
Images Left 59484/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 59486/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 59488/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 59491/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 59494/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 59495/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 59499/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 59500/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 59503/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 59504/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 59505/38455.Ligatures separated successfully. Total ligatures 1.

Images Left 59661/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 59664/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 59668/38455.Ligatures separated successfully. Total ligatures 4.
Images Left 59669/38455.Ligatures separated successfully. Total ligatures 3.
Images Left 59670/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 59673/38455.Ligatures separated successfully. Total ligatures 3.
Images Left 59674/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 59675/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 59676/38455.Ligatures separated successfully. Total ligatures 2.
Images Left 59677/38455.Ligatures separated successfully. Total ligatures 2.
Images Left 59678/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 59679/38455.Ligatures separated successfully. Total ligatures 2.
Images Left 59680/38455.Ligatures separated successfully. Total ligatures 2.

Images Left 59841/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 59842/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 59844/38455.Ligatures separated successfully. Total ligatures 3.
Images Left 59845/38455.Ligatures separated successfully. Total ligatures 3.
Images Left 59847/38455.Ligatures separated successfully. Total ligatures 3.
Images Left 59848/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 59849/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 59850/38455.Ligatures separated successfully. Total ligatures 3.
Images Left 59854/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 59857/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 59858/38455.Ligatures separated successfully. Total ligatures 3.
Images Left 59859/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 59860/38455.Ligatures separated successfully. Total ligatures 1.

Images Left 60022/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 60023/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 60024/38455.Ligatures separated successfully. Total ligatures 3.
Images Left 60025/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 60030/38455.Ligatures separated successfully. Total ligatures 5.
Images Left 60031/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 60032/38455.Ligatures separated successfully. Total ligatures 4.
Images Left 60033/38455.Ligatures separated successfully. Total ligatures 4.
Images Left 60035/38455.Ligatures separated successfully. Total ligatures 2.
Images Left 60036/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 60037/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 60038/38455.Ligatures separated successfully. Total ligatures 1.
Images Left 60039/38455.Ligatures separated successfully. Total ligatures 3.